In [2]:
from presidio_analyzer import AnalyzerEngine
import pandas as pd 
#! python -m spacy download en_core_web_lg
from presidio_analyzer import RecognizerRegistry
#from RecognizerRegistry import email_recognizer
#from presidio_analyzer import predefined_recognizers
#from presidio_analyzer.predefined_recognizers import EmailRecognizer
#python -m spacy download en_core_web_lg
#salt hash

In [3]:
#python -m spacy download en_core_web_lg

In [4]:
%%capture
engine = AnalyzerEngine(default_score_threshold=0, enable_trace_pii=True)


[2020-06-05 13:06:44,452][presidio][INFO]nlp_engine not provided. Creating new SpacyNlpEngine instance
[2020-06-05 13:06:44,727][presidio][INFO]Loading NLP model: spaCy en_core_web_lg
[2020-06-05 13:06:55,996][presidio][INFO]Printing spaCy model and package details:

 {'lang': 'en', 'name': 'core_web_lg', 'license': 'MIT', 'author': 'Explosion', 'url': 'https://explosion.ai', 'email': 'contact@explosion.ai', 'description': 'English multi-task CNN trained on OntoNotes, with GloVe vectors trained on Common Crawl. Assigns word vectors, context-specific token vectors, POS tags, dependency parse and named entities.', 'sources': [{'name': 'OntoNotes 5', 'url': 'https://catalog.ldc.upenn.edu/LDC2013T19', 'license': 'commercial (licensed by Explosion)'}, {'name': 'Common Crawl'}], 'pipeline': ['tagger', 'parser', 'ner'], 'version': '2.2.5', 'spacy_version': '>=2.2.2', 'parent_package': 'spacy', 'accuracy': {'las': 90.1734441725, 'uas': 92.0132337105, 'token_acc': 99.7579930934, 'tags_acc': 97.

# Create Fake data 

In [7]:
from faker import Faker

In [8]:
fake = Faker()
fake.profile()

{'job': 'Research scientist (life sciences)',
 'company': 'Jones-Dennis',
 'ssn': '503-46-3487',
 'residence': '83053 Todd Forest Apt. 718\nWest Kimberlyside, SC 32354',
 'current_location': (Decimal('38.356150'), Decimal('-160.372951')),
 'blood_group': 'O-',
 'website': ['https://mccullough.org/',
  'https://lawson.com/',
  'http://morales-gardner.com/',
  'http://adkins-ford.com/'],
 'username': 'loliver',
 'name': 'Jacob Bartlett',
 'sex': 'M',
 'address': '55475 Tracy Street Suite 749\nHansenmouth, CO 14787',
 'mail': 'ymorris@yahoo.com',
 'birthdate': datetime.date(1991, 6, 12)}

In [9]:
df = pd.DataFrame([fake.profile(), fake.profile(), fake.profile(), fake.profile()])
df

,address,birthdate,blood_group,company,current_location,job,mail,name,residence,sex,ssn,username,website
0,"0159 Joseph Stravenue\nNorth Cassandra, AR 15195",1952-10-20,O-,Oneill LLC,"(-5.7738285, 70.635335)",Operational investment banker,emily52@yahoo.com,Donna Mayo,70964 Villanueva Street Suite 047\nNew Michael...,F,009-14-5245,millerruben,[https://smith.com/]
1,"0582 Garcia Path\nJenniferbury, SD 01512",1962-12-19,AB-,"Brown, Carpenter and Scott","(75.7079875, 120.988975)","Surveyor, rural practice",gvalencia@yahoo.com,William Barnes,"89624 Padilla Camp Suite 481\nWest Elizabeth, ...",M,865-53-0280,melvin06,"[https://park-foster.com/, http://www.clark.biz/]"
2,"8689 Walker Island Apt. 627\nEast Crystalfort,...",2009-10-29,A-,Ayers-Coleman,"(78.784393, -156.462782)",Volunteer coordinator,saustin@gmail.com,Jason Cunningham,"82732 Owens Tunnel\nNorth Amberburgh, FL 73795",M,429-72-3294,michaellopez,"[http://www.clark.com/, https://www.clark.net/..."
3,USNV Carpenter\nFPO AE 88711,1915-10-05,AB+,"Owens, Sanchez and Martin","(33.133429, -162.401107)","Conservation officer, nature",grodriguez@hotmail.com,Michael Oconnor,"637 Proctor Greens\nReginamouth, MI 54825",M,545-27-5969,feliciairwin,"[https://www.fisher.info/, https://www.salazar..."


## Cases
Assumptions:
    - Data comes in as a table
    - The client wants to identify all possible PII. You should never do analytics on it.

1. The client knows which specific columns have PII
    - We can identify each case, redact it, and/or anonymize it

2. The client wants to find out all possible PII
    - We can identify each case, redact it, and/or anonymize it


## Automatic Detection

In [10]:
# import "entities"
from presidio_analyzer.predefined_recognizers import EmailRecognizer
from functools import reduce


In [17]:
text = "rtcrichard@gmail.com "

response = engine.analyze(correlation_id=0,
                          text = text,
                          entities=[],
                          language='en',
                          all_fields=True,
                          score_threshold=0)

for item in response:
    print("Start = {}, end = {}, entity = {}, confidence = {}".format(item.start,
                                                                      item.end,
                                                                      item.entity_type,
                                                                      item.score))

Start = 0, end = 20, entity = EMAIL_ADDRESS, confidence = 1.0
Start = 11, end = 20, entity = DOMAIN_NAME, confidence = 1.0


In [33]:
def identify_pii(text):
    response = engine.analyze(correlation_id=0, text = text, entities=[], language='en', all_fields=True, score_threshold=0.85)    
    return response

In [34]:
response

[type: EMAIL_ADDRESS, start: 46, end: 66, score: 1.0,
 type: DOMAIN_NAME, start: 57, end: 66, score: 1.0,
 type: PERSON, start: 11, end: 16, score: 0.85,
 type: PERSON, start: 68, end: 73, score: 0.85]

## Redact information

In [25]:
text = 'My name is David and I am hungry. My email is rtcrichard@gmail.com, David'
response = identify_pii(text)
print(text)
response

My name is David and I am hungry. My email is rtcrichard@gmail.com, David


[type: EMAIL_ADDRESS, start: 46, end: 66, score: 1.0,
 type: DOMAIN_NAME, start: 57, end: 66, score: 1.0,
 type: PERSON, start: 11, end: 16, score: 0.85,
 type: PERSON, start: 68, end: 73, score: 0.85]

In [67]:
def create_mapping(text):
    response = identify_pii(text)
    word_mapping = {}
    for r in response:
        word_mapping[text[r.start : r.end]] = str("<" + r.entity_type + ">")
    return word_mapping

In [68]:
def redact_info(text):
    word_mapping = create_mapping(text)
    return reduce(lambda a, kv: a.replace(*kv), word_mapping.items(), text)

In [69]:
redact_info(text)

'My name is <PERSON> and I am hungry. My email is <EMAIL_ADDRESS>, <PERSON>'

In [71]:
df_redact = df.applymap(lambda x: redact_info(str(x)))
df_redact

,address,birthdate,blood_group,company,current_location,job,mail,name,residence,sex,ssn,username,website
0,"<DATE_TIME> <PERSON> Village\n<LOCATION>, AZ 2...",<DATE_TIME>,O-,<PERSON>,"(<LOCATION>'), Decimal('82.999805'))",Learning disability nurse,<EMAIL_ADDRESS>,<PERSON>,USCGC Sanchez\nFPO AA 40181,F,585-01-1007,mhaynes,['https://<DOMAIN_NAME>/']
1,"638 Wells Prairie\n<LOCATION>, OR 89816",<DATE_TIME>,A-,<PERSON>,"(Decimal('-55.162483'), <PERSON>'))",Science writer,<EMAIL_ADDRESS>,<PERSON>,"36617 Leah Locks\n<PERSON>, <LOCATION> 69812",M,561-13-4352,ldavidson,"['https://<DOMAIN_NAME>/', 'http://<DOMAIN_NAM..."
2,"973 John Spring Suite 959\n<PERSON>, IA 26014",<DATE_TIME>,O+,<PERSON>,"(Decimal('-51.762098'), Decimal('-122.061227'))",Research scientist (medical),<EMAIL_ADDRESS>,<PERSON>,"18892 Cobb Ridges Apt. 843\n<LOCATION>, KY 76030",F,431-41-6297,carolynbrown,"['https://<DOMAIN_NAME>/', 'http://<DOMAIN_NAM..."
3,"<PERSON>\n<PERSON>, <DATE_TIME>",<DATE_TIME>,O+,<PERSON>,"(Decimal('55.204783'), Decimal('108.359621'))",Pension scheme manager,<EMAIL_ADDRESS>,<PERSON>,"46252 <LOCATION><LOCATION>, <LOCATION> <DATE_T...",M,497-42-0685,wilcoxsarah,"['https://<DOMAIN_NAME>/', 'https://<DOMAIN_NA..."


## Anonymize data using Faker

In [76]:
word_mapping = create_mapping(text)
word_mapping

{'rtcrichard@gmail.com': '<EMAIL_ADDRESS>',
 'gmail.com': '<DOMAIN_NAME>',
 'David': '<PERSON>'}

In [78]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_extraction.text import HashingVectorizer
from collections import defaultdict


In [79]:
def anonymize_name(data):
    names  = defaultdict(fake.name)
    for d in data:
        names[d]
    return names


In [80]:
x = anonymize_name(['richard','bob', 'anna'])
x

defaultdict(<bound method Provider.name of <faker.providers.person.en_US.Provider object at 0x7f8e838f8da0>>,
            {'richard': 'Derrick Mitchell',
             'bob': 'Matthew Powers',
             'anna': 'William Garza'})

In [81]:
fake.email()

'rochaangela@hotmail.com'

In [82]:
df = pd.DataFrame([fake.profile(), fake.profile(), fake.profile(), fake.profile()])
df.head(1)

,address,birthdate,blood_group,company,current_location,job,mail,name,residence,sex,ssn,username,website
0,"971 Summers Mountains\nMarshallland, OH 06583",2008-10-24,O-,Mills-Hanson,"(77.601293, -132.151889)",Hospital pharmacist,mcdonaldbarbara@yahoo.com,Joe Miller,"4678 Andrews Stream Apt. 082\nCooleyhaven, NH ...",M,600-14-7323,khunter,"[http://bishop.com/, http://www.lowery.com/, h..."


In [ ]:
##### Notes from david
#sampele portion of df and run test if cells in each column is an infotype 
# plugin to pandas. df.infotypes 
# defin infotype as classes

# anonymize: map each senstive values to a hex id. 
#second level. Take the recoded value and generate fake data for the anonymized data
#redaction 
#set up with abstract framework. plugins 

## Pipe predefined recopgnizers 

In [89]:
from presidio_analyzer.predefined_recognizers import EmailRecognizer
from presidio_analyzer import entity_recognizer

In [90]:
result = engine.analyze(text='rtcr@gmail.com', language='en', all_fields=True)

In [91]:
result

[type: EMAIL_ADDRESS, start: 0, end: 14, score: 1.0,
 type: DOMAIN_NAME, start: 5, end: 14, score: 1.0]

In [92]:
class InfoType(object):
    """Base infotype"""
    
    def __init__(self):
        """A class to test for infotypes

        Args:
            name: The name for the desired infotype 
        """
        #self.__text__ = text
        #self.__entities__ = entities
        #self.__all_fields__ = all_fields
        #self._identifier_ = identifier

        
    @property
    def text(self):
        """Trained topic model"""
        return self._text

    @property
    def entities(self):
        """Type of model which either already has been or will be trained"""
        return self._entities
    
    @property
    def identifier(self):
        """Type of model which either already has been or will be trained"""
        return self._identifier
    @property
    def all_fields(self):
        """Type of model which either already has been or will be trained"""
        return self._all_fields



    def detector(self, text, all_fields, identifier='presidio', entities=[]):
        """Test if a given value contains an infotype

        Args:
            value: The value to be tested
        """
        self._text = text
        self._identifier = identifier
        self._entities = entities
        self._all_fields = all_fields


        if self.identifier == 'presidio':
            result = engine.analyze(text=self.text, all_fields=self.all_fields, correlation_id=0, language='en', entities=self.entities) 
        else:
            result = "Nope"
        return result

In [93]:
InfoType().detector(text='rtcrichard@gmail.com', all_fields=True, identifier='presidio')

[type: EMAIL_ADDRESS, start: 0, end: 20, score: 1.0,
 type: DOMAIN_NAME, start: 11, end: 20, score: 1.0]

In [94]:
df.head(1)

,address,birthdate,blood_group,company,current_location,job,mail,name,residence,sex,ssn,username,website
0,"971 Summers Mountains\nMarshallland, OH 06583",2008-10-24,O-,Mills-Hanson,"(Decimal('77.601293'), Decimal('-132.151889'))",Hospital pharmacist,mcdonaldbarbara@yahoo.com,Joe Miller,"4678 Andrews Stream Apt. 082\nCooleyhaven, NH ...",M,600-14-7323,khunter,"['http://bishop.com/', 'http://www.lowery.com/..."


In [95]:
e = InfoType()
df['mail'].map(lambda x: e.detector(text=x, all_fields='EmailRecognizer', identifier='presidio') )

0    [type: EMAIL_ADDRESS, start: 0, end: 25, score...
1    [type: EMAIL_ADDRESS, start: 0, end: 19, score...
2    [type: EMAIL_ADDRESS, start: 0, end: 24, score...
3    [type: EMAIL_ADDRESS, start: 0, end: 20, score...
Name: mail, dtype: object

In [96]:
def guess_series_infotype(series, frac=0.5):
    e = InfoType()
    sampled_data = series.sample(frac=frac, random_state=1)
    results = sampled_data.map(lambda x: e.detector(text=x, all_fields='EmailRecognizer', identifier='presidio'))
    #return set([p.entity_type for parts in results for p in parts])
    return results.tolist()


In [97]:
guess_series_infotype(df['mail'])

[[type: EMAIL_ADDRESS, start: 0, end: 20, score: 1.0,
  type: DOMAIN_NAME, start: 9, end: 20, score: 1.0],
 [type: EMAIL_ADDRESS, start: 0, end: 24, score: 1.0,
  type: DOMAIN_NAME, start: 15, end: 24, score: 1.0]]

## type hints 

In [98]:
from typing import NewType, Dict


In [99]:
NAME = NewType('NAME', str)

class Name(Dict):
    person: NAME

In [100]:
type(NAME('rich'))

str

In [101]:
def test_func(a: Name) -> NAME:
    return a
    

In [102]:
hash('hi')

3259049388444818814

In [103]:
hash('bobby')

7261700005794478416